The purpose of this file is to create a graph for Dan (Dauer lab) to use in a grant.

The figure that I create should express the percent of trials with reach performed out of the total trials available.

I will average based on training day.

In [1]:
import thesis_database

project_name = 'hello'
experiment_name='skilled-reaching'
thesis_database.initialize_database(project_name)
experiment = thesis_database.Experiment.from_db(experiment_name = experiment_name)

# Step One: Get a list of all of the scored csv files for all reviewers
import glob
import csv
from pathlib import Path

In [4]:
all_scored_csv = list()
for reviewer_id in thesis_database.tools.get.list_reviewer_ids():
    reviewer = thesis_database.Reviewer.from_db(reviewer_id=reviewer_id)
    this_reviewer_scored_csvs = glob.glob(f"{reviewer.scored_dir}/*.csv")
    for a_scored_csv in this_reviewer_scored_csvs:
        csv_filename = a_scored_csv.split('/')[-1]
        blind_name = csv_filename.split('_')[0]
        with open(a_scored_csv) as f:
            csv_reader = csv.reader(f)
            next(csv_reader)
            for item in csv_reader:
                if len(item) < 4:
                    print(a_scored_csv)
                    continue
                elif len(item) > 4:
                    item = item[:4]
                blind_trial_num, reach_score, abnormal_movt_score, grooming_score = item
                if len(blind_trial_num) < 2:
                    blind_trial_num = f'0{blind_trial_num}'
                full_path = f"{reviewer.toScore_dir}/{blind_name}/{blind_name}_R{blind_trial_num}.mp4"
                all_full_paths = thesis_database.tools.get.list_all_blinded_trial_full_paths_for_reviewer_experiment(reviewer, experiment)
                for path in all_full_paths:
                    if full_path in path:
                        blind_trial = thesis_database.BlindTrial.from_db(full_path=path)
                        trial = thesis_database.Trial.from_db(trial_id=blind_trial.trial_id)
                        trial_num = int( Path(trial.trial_dir).stem.split('_R')[-1] )
                        if len(reach_score) < 1:
                            print(path)
                            break
                        thesis_database.TrialScore(trial.trial_id, reviewer.reviewer_id, int(trial_num), int(reach_score),
                                                   bool(abnormal_movt_score), bool(grooming_score)).save_to_db()
    all_scored_csv.extend(this_reviewer_scored_csvs)


/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/dason1tylh/dason1tylh_R43.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/dason1tylh/dason1tylh_R44.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/dason1tylh/dason1tylh_R45.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/jwxud1qywq/jwxud1qywq_R44.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/jxrr8n27ck/jxrr8n27ck_R43.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/xu8imx3s7j/xu8imx3s7j_R01.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/xu8imx3s7j/xu8imx3s7j_R02.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/xu8imx3s7j/xu8imx3s7j_R03.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/xu8imx3s7j/xu8imx3s7j_R04.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/xu8imx3s7j/xu8imx3s7j_R05.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/xu8imx3s7j/xu8imx3s7j_R06.mp4
/Users/Krista/Desktop/blindScoring/Krista_K/toScore_KK/xu8imx3s7j

In [19]:
# Add session numbers from session_dir
for session_dir in thesis_database.tools.get.list_all_session_dir_for_experiment(experiment):
    session = thesis_database.Session.from_db(session_dir=session_dir)
    session_dir_split = session_dir.split('T')
    if len(session_dir_split) > 3:
        session_num = session_dir_split[-2].split('-')[0]
    else:
        session_num = session_dir_split[-1]
    session.session_num = session_num
    session.save_to_db()